In [9]:
import json
import requests
import math
import geopandas as gpd
import pandas as pd

In [157]:
f = open('/Users/friederikebauer/Developer/post-covid-metadatenkatalog/mii-testdaten/Bundle1.json',)
data = json.load(f)

In [ ]:
# todo: do I need Address class
class Address: 
    def __init__(self):
        country: str | None = None
        federal_state: str | None = None
        city: str | None = None
        postal_code: str | None = None
        street: str | None = None

# todo: oder Klasse definieren wie wir denken es kommt raus aus den Daten? Hier eher Ziel-Datenstruktur definiert
class Patient: 
    def __init__(self):
        patient_id: str | None = None
        birth_date: str | None = None
        # address
        country: str | None = None
        federal_state: str | None = None
        city: str | None = None
        postal_code: str | None = None
        street: str | None = None
        encounter: str | None = None  #todo: make encounter class?
        condition: str | None = None



class PatientCollection: 
    # todo: method um alle Zusatz-Infos zu extrahieren?
    # todo: macht es sinn eine PatientCollection klasse zu haben?
    def __init__(self):
        self._patients: list[Patient] = []


    def extract_patients(self, data):
        patients: list[Patient] = []

        # filter out patients + their relevant data + set up initial patients collection
        for el in data["entry"]:
            resource = el["resource"]["resourceType"]
            
            if resource == "Patient":
                patient = Patient()
                try: 
                    patient.patient_id = el["resource"]["id"]
                    patient.birth_date = el["resource"]["birthDate"]

                except KeyError: 
                    print("Error: Basic Patient data incomplete")
                    continue
                
                # add address data
                try: 
                    patient.country =  el["resource"]["address"][0].get("country", None)  # returns country and if country not there returns None
                    patient.federal_state = el["resource"]["address"][0].get("state", None)
                    patient.city = el["resource"]["address"][0].get("city", None)
                    patient.postal_code = el["resource"]["address"][0].get("postalCode", None)

                except KeyError: 
                    print("Error: Patient Address data incomplete")
                    continue

                # patient street is accessed differently as it is mostly not given -> if it fails all other address elements fail
                try: 
                    patient.street = el["resource"]["address"][0]["line"][0]
                except KeyError: 
                    print("Error: Patient Street data incomplete")
                    continue

                
                
                self._patients.append(patient)

    def extract_patient_encounter_info(self, data):
        for el in data["entry"]:

            # ENCOUNTER
            if el["resource"]["resourceType"] == "Encounter":
                encounter_patient_id = el["resource"]["subject"]["reference"].split("/")[1] # string = Patient/{id}
                for patient in self._patients: # find patient with id + add info to him
                    if patient.patient_id == encounter_patient_id:
                        try: 
                            patient.encounter = el["resource"]["type"][0]["coding"][0]["display"]
                        except: 
                            print(el["resource"])
                            print("Encounter data incomplete")
                            continue


    # todo: sind wir auch an den ids der conditions interessiert?
    def extract_patient_condition(self, data):
        for el in data["entry"]:

            # CONDITION
            if el["resource"]["resourceType"] == "Condition":
                condition_patient_id = el["resource"]["subject"]["reference"].split("/")[1] # string = Patient/{id}
                for patient in self._patients:
                    if patient.patient_id == condition_patient_id:
                        try: 
                            patient.condition = el["resource"]["code"]["coding"][0]["display"]
                        except: 
                            print("Condition data incomplete")
                            continue


    # allow direct access to patients           
    def __getitem__(self, index):
        return self._patients[index]
        
    def __len__(self):
        return len(self._patients)
        
# todo: macht es Sinn eine Patients klasse zu bauen? und dann dort die methode um patients zu extrahieren


In [163]:
# init PatientCollection + access patients
patients = PatientCollection()
patients.extract_patients(data)
patients.extract_patient_encounter_info(data)
patients.extract_patient_condition(data)

Error: Patient Street data incomplete
{'resourceType': 'Encounter', 'id': '591', 'meta': {'versionId': '1', 'lastUpdated': '2024-11-07T08:02:52.619+00:00', 'source': '#aI3qAal5bS5VepYV', 'profile': ['https://www.medizininformatik-initiative.de/fhir/core/modul-fall/StructureDefinition/KontaktGesundheitseinrichtung']}, 'status': 'unknown', 'class': {'system': 'http://terminology.hl7.org/CodeSystem/v3-ActCode', 'code': 'IMP', 'display': 'inpatient encounter'}, 'subject': {'reference': 'Patient/590'}, 'period': {'start': '2023-06-14'}}
Encounter data incomplete
{'resourceType': 'Encounter', 'id': '596', 'meta': {'versionId': '1', 'lastUpdated': '2024-11-07T08:02:52.619+00:00', 'source': '#aI3qAal5bS5VepYV', 'profile': ['https://www.medizininformatik-initiative.de/fhir/core/modul-fall/StructureDefinition/KontaktGesundheitseinrichtung']}, 'status': 'unknown', 'class': {'system': 'http://terminology.hl7.org/CodeSystem/v3-ActCode', 'code': 'IMP', 'display': 'inpatient encounter'}, 'subject': {

In [ ]:
# accessing patients direcly
patients[1].__dict__


{'patient_id': '595',
 'birth_date': '1980-05-18',
 'country': 'DE',
 'federal_state': 'DE-BE',
 'city': 'Berlin',
 'postal_code': '10435',
 'street': 'Wollinerstr. 5',
 'condition': 'COVID-19, virus identified'}

# Luftdaten - API
- check: Daten bevor 2016 nicht verfügbar

In [ ]:
# 1) get all stations

# 2) calculate closest station to patient home address

# 3) request data on closest stations

In [ ]:
# 1) GET ALL STATIONS

# Define the URL for the API endpoint
# todo: check again from where we want the stations (date, time, etc, )
url = "https://www.umweltbundesamt.de/api/air_data/v3/stations/json?use=airquality&lang=de&date_from=2019-01-01&date_to=2019-01-01&time_from=9&time_to=9"

response = requests.get(url)
response_data = response.json()
len(response_data["data"])

In [186]:
stations = []

for key, value in response_data["data"].items():
    stations.append({
    "id": value[0],
    "code": value[1],
    "name": value[2], 
    "longitude": value[7], 
    "latitude": value[8]})

In [ ]:
# 2) CALCULATE CLOSEST STATION TO PATIENT HOME ADDRESS


# todo: add imlementation with haversine directly from this package: https://pypi.org/project/haversine/

def get_distance_from_lat_lon(lat_location, lon_location, lat_station, lon_station):
    R = 6371  # Radius of the Earth in km
    # Convert degrees to radians
    dLat = math.radians(lat_station - lat_location)
    dLon = math.radians(lon_station - lon_location)
    
    # Haversine formula
    a = (math.sin(dLat / 2) ** 2 + 
         math.cos(math.radians(lat_location)) * math.cos(math.radians(lat_station)) * 
         math.sin(dLon / 2) ** 2)
    
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    distance = R * c # Distance in km
    return distance


def get_closest_station(lat, long):
    # function uses the haverstine calculation
    closest_station = None

    for station in stations: 
        station_lat = float(station["latitude"])
        station_long = float(station["longitude"])

        distance = get_distance_from_lat_lon(lat, long, station_lat, station_long)
        if closest_station is None or distance < closest_station["distance"]:
            closest_station = {
                "station": station,
                "distance": distance
            }
    return closest_station

# closest = get_closest_station(lat= 48.808660, long=9.220560) # stuttgart - bad cannstatt
closest = get_closest_station(lat = 52.9357426, long = 13.4748363) # pasteurstrasse - berlin
closest

{'station': {'id': '58',
  'code': 'DEBB060',
  'name': 'Eberswalde, Breite Straße',
  'longitude': '13.8222',
  'latitude': '52.8325'},
 'distance': 25.981318987891243}

In [205]:
#3) REQUEST DATA ON CLOSEST STATIONS
# what are we interested in?
# - time-frame
# do we want the time series? the mean value? the median?
# -> add options to the function to calculate different values


# todo: we will disregard time in the day? -> calculcate back 6 months, to the front 6 months

# it returns air quality index
def get_station_index(station_id: str, start_date: str, end_date: str):
    request_url = f"https://www.umweltbundesamt.de/api/air_data/v3/airquality/json?date_from={start_date}&time_from=1&date_to={end_date}&time_to=24&station={station_id}"
    response = requests.get(request_url)
    response_data = response.json()
    
    return response_data


# components = Schadstoffe, z.B. Ozon (id=3)
# scope = z.B. Tagesmittel (id=1)
def get_station_data(station_id: str, start_date: str, end_date: str, component_id: int, scope_id: int):
    request_url = f"https://www.umweltbundesamt.de/api/air_data/v3/measures/json?date_from={start_date}&time_from=1&date_to={end_date}&time_to=24&station={station_id}&component={component_id}&scope={scope_id}"
    response = requests.get(request_url)
    response_data = response.json()
    return response_data




In [206]:
# AIR QUALITY TRIAL
closest_station_index = get_station_index(station_id = closest["station"]["id"], start_date =  "2019-01-01", end_date = "2019-01-01")
# closest_station_index["data"] 
# -> todo: gives me back hourly data for the day

In [ ]:
# AIR DATA TRIAL
closest_station_data = get_station_data(station_id = closest["station"]["id"], 
                                        start_date = "2019-01-01", 
                                        end_date = "2019-01-01", 
                                        component_id = 1, 
                                        scope_id = 1)

closest_station_data
# todo: nicht für alle component_id und scope_id gibt es daten -> abfangen wenn es keine Daten gibt

{'request': {'component': '1',
  'scope': '1',
  'station': '58',
  'date_from': '2019-01-01',
  'date_to': '2019-01-01',
  'time_from': '01:00:00',
  'time_to': '24:00:00',
  'index': 'id',
  'lang': 'en',
  'datetime_from': '2019-01-01 00:00:00',
  'datetime_to': '2019-01-01 23:00:00'},
 'indices': {'data': {'station id': {'date start': ['0: Id of component - integer',
     '1: Id of scope - integer',
     '2: Value - number',
     '3: Date of measure end - string',
     '4: Index - string|null']}}},
 'data': {'58': {'2019-01-01 11:00:00': [1,
    1,
    10,
    '2019-01-01 12:00:00',
    '1']}}}

In [208]:
closest["station"]["id"]

'58'

# Data exploration

In [ ]:
# WELCHE DATEN SIND RELEVANT? WELCHE DATEN SIND VORHANDEN?

for el in data["entry"]:
    resource = el["resource"]["resourceType"]
    if resource == "Patient":
        pass
        # print(el["resource"])
        '''
        address: 
        - city
        - postal code
        - country
        - state
        '''
        # straße -> 'type': 'both', 'line': ['Berliner Str. 1'], 
        # abkürzungen bundesland -> https://www.destatis.de/DE/Methoden/abkuerzung-bundeslaender-DE-EN.html

    elif resource == "Encounter":
        print(el["resource"])
        # pass
        # print(el["resource"])
        # subject["reference"] enthält 'Patient/XXX'
        # id, meta, status, class, type, subject, period

    elif resource == "Procedure":
        # status = [completed, in-progress]
        # id mapping to patient: subject["reference"] enthält 'Patient/XXX'
        # id, meta, status, category, code, subject, performedDateTime, bodySite, note
        # note["text"] shows additional information
        pass
        # print(el["resource"])

    elif resource == "Condition":
        # print(el["resource"])
        # id mapping to patient: subject["reference"] enthält 'Patient/XXX'
        pass

    elif resource == "MedicationStatement":
        print(el["resource"])
        pass

    else: raise ValueError("New Ressource Type")


'''resourceType(s):
- Patient
- Encounter
- Procedure
- Condition
- MedicationStatement
'''

{'resourceType': 'Encounter', 'id': '585', 'meta': {'versionId': '1', 'lastUpdated': '2024-11-07T08:02:52.619+00:00', 'source': '#aI3qAal5bS5VepYV', 'profile': ['https://www.medizininformatik-initiative.de/fhir/core/modul-fall/StructureDefinition/KontaktGesundheitseinrichtung']}, 'status': 'in-progress', 'class': {'system': 'http://terminology.hl7.org/CodeSystem/v3-ActCode', 'code': 'IMP', 'display': 'inpatient encounter'}, 'type': [{'coding': [{'system': 'http://snomed.info/sct', 'code': '3390906007', 'display': 'Follow-up encounter '}]}], 'subject': {'reference': 'Patient/584'}, 'period': {'start': '2022-10-01T10:00:00Z'}}
{'resourceType': 'MedicationStatement', 'id': '589', 'meta': {'versionId': '1', 'lastUpdated': '2024-11-07T08:02:52.619+00:00', 'source': '#aI3qAal5bS5VepYV', 'profile': ['https://www.medizininformatik-initiative.de/fhir/core/modul-medikation/StructureDefinition/MedicationStatement']}, 'status': 'active', 'medicationCodeableConcept': {'coding': [{'system': 'http://

'resourceType(s):\n- Patient\n- Encounter\n- Procedure\n- Condition\n- MedicationStatement\n'